### Prepping Data Challenge:  Elden Ring Adventure (week 37)

### Requirements
- Load the dataset - updated 14/09
- Split the dataset into two tables
     - Damage Stats: 1 containing: Name, Category, Phy, Mag, Fire, Ligh, Holy
     - Level Requirements: 1 containing: Name, Str, Dex, Int, Fai, Arc
- For Damage Stats, if you look at the data in Phy, Mag, Fire, Ligh, Holy:
     - the first value shows the attack damage 
     - the second value shows the damage resistance
     - A dash “-” means 0, i.e. no damage 
- For Level Requirements, for Str, Dex, Int, Fai, Arc:
     - the first value shows the level required
     - the second value shows the weapon scaling rating
     - A dash “-” means 0, i.e. no requirement or scaling 
- Pivot the two datasets:
     - For Damage Stats,1 column containing Phy, Mag, Fire, Ligh, Holy and 1 for the values
     - For Level Requirements,1 column containing Str, Dex, Int, Fai, Arc and 1 for the values
- Split the pivot values into 2 columns for both tables:
     - For Damage Stats label the first column “Attack Damage”, and the second “Damage Resistance”
     - For Level Requirements label the first column “Required Level”, and the second “Attribute Scaling” 
- Replace the “-” values with zero in the columns: “Attack Damage”, “Damage Resistance”, “Required Level” and change the datatype to whole numbers (integer)
- Find the total “Attack Damage” and total “Required Level” for all weapons and join the datasets together.
- Rank the weapons by total attack damage, grouped by the total required attribute. 
- Filter for the number 1 rank and output the data

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Load the datasets
df =  (pd.read_csv(r"\Dataprep\2022\elden_ring_combat_weapons_raw.csv", 
                   usecols=['Name','Category'] + 
                   ['Phy', 'Mag', 'Fire', 'Ligh', 'Holy'] +
                   ['Str', 'Dex', 'Int', 'Fai', 'Arc'])
         .melt(id_vars=['Name','Category'], var_name='orig_col', value_name='orig_value'))

In [3]:
df.head(10)

,Name,Category,orig_col,orig_value
0,Dagger,Daggers,Phy,74 35
1,Parrying Dagger,Daggers,Phy,75 35
2,Misericorde,Daggers,Phy,92 36
3,Great Knife,Daggers,Phy,75 35
4,Bloodstained Dagger,Daggers,Phy,81 36
5,Erdsteel Dagger,Daggers,Phy,67 36
6,Wakizashi,Daggers,Phy,94 42
7,Celebrant's Sickle,Daggers,Phy,79 35
8,Ivory Sickle,Daggers,Phy,60 26
9,Crystal Knife,Daggers,Phy,82 32


In [4]:
# - Split the dataset into two tables
df[['val1', 'val2']] = (df['orig_value']
    .str.replace(r'[^\s\d]', '0', regex=True)  # Replace non-digit and non-space characters with '0'
    .str.extract(r'(\S+)\s+(\S+)').astype(int))

In [5]:
# Split the pivot values into 2 columns for both tables:
# Replace the “-” values with zero in the columns: “Attack Damage”, “Damage Resistance”, “Required Level” and change the datatype to whole numbers (integer)
# Find the total “Attack Damage” and total “Required Level” for all weapons and join the datasets together.

DAMAGE_RENAMES = {'val1' : 'Attack Damage', 'val2' : 'Damage Resistance' }
LEVEL_RENAMES = {'val1' : 'Required Level', 'val2' : 'Attribute Scaling' }

df2 = ( df.melt(id_vars=['Name', 'Category'] + ['orig_col'], value_vars=['val1', 'val2'], var_name='value_type')
          .assign(new_col_name = lambda df_x: np.where(df_x['orig_col'].isin(['Phy', 'Mag', 'Fire', 'Ligh', 'Holy']), 
                                                    df_x['value_type'].replace(DAMAGE_RENAMES),
                                                    df_x['value_type'].replace(LEVEL_RENAMES)))
          .pivot_table(index=['Name', 'Category'], columns='new_col_name', values='value', aggfunc='sum')
          .reset_index() )

In [6]:
df2.head()

new_col_name,Name,Category,Attack Damage,Attribute Scaling,Damage Resistance,Required Level
0,Alabaster Lord's Sword,Greatswords,166,0,170,46
1,Antspur Rapier,Thrusting Swords,98,0,171,30
2,Axe of Godfrey,Colossal Weapons,165,0,255,56
3,Axe of Godrick,Greataxes,142,0,205,56
4,Bandit's Curved Sword,Curved Swords,118,0,192,24


In [7]:
#- Rank the weapons by total attack damage, grouped by the total required attribute. 
# Filter for the number 1 rank and output the data
df2['Rank'] = df2.groupby('Required Level')['Attack Damage'].rank(ascending=False, method='min')

In [8]:
output = df2[['Name', 'Category','Required Level', 'Attack Damage']]

In [9]:
output.head(20)

new_col_name,Name,Category,Required Level,Attack Damage
0,Alabaster Lord's Sword,Greatswords,46,166
1,Antspur Rapier,Thrusting Swords,30,98
2,Axe of Godfrey,Colossal Weapons,56,165
3,Axe of Godrick,Greataxes,56,142
4,Bandit's Curved Sword,Curved Swords,24,118
5,Banished Knight's Greatsword,Greatswords,26,142
6,Banished Knight's Halberd,Halberds,26,125
7,Bastard Sword,Greatswords,26,138
8,Bastard's Stars,Flails,52,143
9,Battle Axe,Axes,20,123


In [10]:
#output the data
output.to_csv('wk36-output.csv', index=False)